In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

     |████████████████████████████████| 320.4 MB 33 kB/s  eta 0:00:01    |███████                         | 70.7 MB 518 kB/s eta 0:08:02     |███████████                     | 109.2 MB 612 kB/s eta 0:05:45     |███████████                     | 110.6 MB 612 kB/s eta 0:05:43     |███████████████████████         | 231.4 MB 7.2 MB/s eta 0:00:13     |████████████████████████▎       | 243.2 MB 7.0 MB/s eta 0:00:12     |█████████████████████████▎      | 253.1 MB 640 kB/s eta 0:01:46     |█████████████████████████▌      | 255.0 MB 640 kB/s eta 0:01:42
     |████████████████████████████████| 26.1 MB 207 kB/s eta 0:00:01
     |████████████████████████████████| 459 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 20.1 MB 516 kB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-est

In [2]:
import gym
import random

In [3]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
actions

2

In [5]:
states

4

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [9]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [32]:
del model

In [23]:
model = build_model(states, actions)

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [25]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [26]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy = policy, nb_actions = actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [27]:
dqn = build_agent(model,actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000,visualize=False,verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 4:47 - reward: 1.0000

/home/swarm/anaconda3/envs/env/lib/python3.6/site-packages/rl/memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 50s 5ms/step - reward: 1.0000
109 episodes - episode_reward: 91.661 [10.000, 200.000] - loss: 3.119 - mae: 19.198 - mean_q: 38.873

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 197.960 [157.000, 200.000] - loss: 8.160 - mae: 40.832 - mean_q: 82.510

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 199.520 [184.000, 200.000] - loss: 9.021 - mae: 44.608 - mean_q: 89.889

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 48s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 199.060 [171.000, 200.000] - loss: 9.986 - mae: 43.906 - mean_q: 88.082

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 51s 5ms/step - reward: 1.0000
done, took 244.694 seconds


In [34]:
scores = dqn.test(env,nb_episodes=100,visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [33]:
_ = dqn.test(env, nb_episodes=5, visualize = True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
